In [13]:
import pandas as pd
from typing import List, Tuple, Dict, NamedTuple

# Read Code

In [140]:
#%%writefile preprocess.py
from typing import List
from datetime import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd



PATH_TO_DATA = 'gs://churn_prediction_inputs/rawdata/curn_prediction_rawdata.csv'
DATE_COL = 'ActiveDate'
USER_ID_COL = 'ID'
TARGET_COL = 'target'
ALL_EQUITY_COLUMN = 'EOM_Equity'
DATES_COLUMNS = ['ActiveDate', 'FTDdate', 'LastPosOpenDate', 'LastLoggedIn']


def convert_objects_to_dates(rawdata:pd.DataFrame, date_columns: List[str] = DATES_COLUMNS) -> pd.DataFrame:
    """
    This function convert dates from object type to date type column inplae
    """
    for date_column in date_columns:
        rawdata[date_column] = pd.to_datetime(rawdata[date_column])
    
    return rawdata


def exclude_users_with_gaps_in_time_points(rawdata: pd.DataFrame) -> pd.DataFrame:
    """
    This function find the user with time gaps and and exclude them form
    the data
    """
    rawdata = rawdata.sort_values(by=[USER_ID_COL, DATE_COL], ascending=False)
    
    # filter useres that have no times gaps
    total_number_of_time_points = rawdata[DATE_COL].nunique()
    users_time_points = rawdata.groupby(USER_ID_COL)[DATE_COL].nunique()
    users_with_potential_gaps = users_time_points[
        users_time_points < total_number_of_time_points].index
    
    users_with_potential_gaps_df = rawdata[rawdata[USER_ID_COL].isin(users_with_potential_gaps)].groupby(USER_ID_COL)
    
    users_to_exclude = set()
    for user_id, user_data in users_with_potential_gaps_df:            
        month_first_record = None
        for index, temp_date in enumerate(user_data[DATE_COL].dt.strftime('%Y-%m')):
            temp_date = datetime.strptime(temp_date, "%Y-%m")

            if index==0:
                month_first_record = temp_date

            elif temp_date+relativedelta(months=index)!=month_first_record:
                users_to_exclude.add(user_id)
    
    return users_to_exclude

                    
def generte_target(rawdata: pd.DataFrame, months_ahead: int = 1) -> pd.DataFrame:
    """
    This function generate the super visor (the target feature)
    """
    # find the follwing next EOM Equity amount 
    rawdata = rawdata.sort_values(by=[USER_ID_COL, DATE_COL], ascending=False)
    rawdata[f'next_{ALL_EQUITY_COLUMN}'] = rawdata.groupby([USER_ID_COL])[ALL_EQUITY_COLUMN].shift(months_ahead)
    
    # filter the last observation because we cont have the lead value (target)
    rawdata = rawdata[~rawdata[f'next_{ALL_EQUITY_COLUMN}'].isna()]
    
    # defined the target variable
    rawdata[TARGET_COL] = np.where(
        (rawdata[f'next_{ALL_EQUITY_COLUMN}'] < 25) & 
        (rawdata[ALL_EQUITY_COLUMN]>=25), 1, 
        np.where(
            (rawdata[f'next_{ALL_EQUITY_COLUMN}'] < 25) & 
            (rawdata[ALL_EQUITY_COLUMN]<25), -1, 0)
    )
    
    return rawdata


def preprocess(path_to_data: str = PATH_TO_DATA):
    """
    """
    # read data
    rawdata = pd.read_csv(path_to_data)
    rawdata.dropna(axis=0, how="all", inplace=True)
    
    # change features types
    rawdata = convert_objects_to_dates(rawdata)
    
    # exclude users with missing time points
    exclude = exclude_users_with_gaps_in_time_points(rawdata)
    rawdata = rawdata[~rawdata[USER_ID_COL].isin(exclude)]
    
    # generate target variable
    rawdata = generte_target(rawdata)
    
    # filter all non relevant rows
    rawdata = rawdata[(~rawdata[TARGET_COL].isna()) &
                      (rawdata[TARGET_COL]>=0)]
    
    return rawdata

# Test functions

In [141]:
%%time
rawdata = preprocess()

/opt/conda/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (0,1,3,5,6,7,8,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 3.73 s, sys: 384 ms, total: 4.12 s
Wall time: 6.71 s


In [124]:
rawdata.target.value_counts()

 0    173181
-1     73102
 1     10215
Name: target, dtype: int64

In [110]:
rawdata = convert_objects_to_date(rawdata)
exclude = exclude_users_with_gaps_in_time_points(rawdata)
rawdata = rawdata[~rawdata[USER_ID_COL].isin(exclude)]
rawdata = generte_target(rawdata)


In [111]:
rawdata

ID ActiveDate  Seniority    FTDdate  \
217755  FFFEBEDA-0DA4-EA11-A2D7-0017A4770404 2021-02-28        8.0 2020-06-16   
217753  FFFEBEDA-0DA4-EA11-A2D7-0017A4770404 2021-01-31        7.0 2020-06-16   
217754  FFFEBEDA-0DA4-EA11-A2D7-0017A4770404 2020-12-31        6.0 2020-06-16   
217752  FFFEBEDA-0DA4-EA11-A2D7-0017A4770404 2020-11-30        5.0 2020-06-16   
219842  FFFEBEDA-0DA4-EA11-A2D7-0017A4770404 2020-10-31        4.0 2020-06-16   
...                                      ...        ...        ...        ...   
277643  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-10-31        4.0 2020-06-06   
277642  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-09-30        3.0 2020-06-06   
277640  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-08-31        2.0 2020-06-06   
284028  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-07-31        1.0 2020-06-06   
277641  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-06-30        0.0 2020-06-06   

         FTDA      Region               Country             Channel  \
217755  200.0      French                France  Introducing Agents   
217753  200.0      French                France  Introducing Agents   
217754  200.0      French                France  Introducing Agents   
217752  200.0      French                France  Introducing Agents   
219842  200.0      French                France  Introducing Agents   
...       ...         ...                   ...                 ...   
277643  400.0  Arabic GCC  United Arab Emirates  Mobile Acquisition   
277642  400.0  Arabic GCC  United Arab Emirates  Mobile Acquisition   
277640  400.0  Arabic GCC  United Arab Emirates  Mobile Acquisition   
284028  400.0  Arabic GCC  United Arab Emirates  Mobile Acquisition   
277641  400.0  Arabic GCC  United Arab Emirates  Mobile Acquisition   

       FirstInstrument  V3_Complete  ... TotalDeposits CountDeposits  \
217755         XNG/USD          1.0  ...           0.0           0.0   
217753         XNG/USD          1.0  ...           0.0           0.0   
217754         XNG/USD          1.0  ...           0.0           0.0   
217752         XNG/USD          1.0  ...           0.0           0.0   
219842         XNG/USD          1.0  ...           0.0           0.0   
...                ...          ...  ...           ...           ...   
277643     Alteneiji80          1.0  ...       11000.0           2.0   
277642     Alteneiji80          1.0  ...       15000.0           1.0   
277640     Alteneiji80          1.0  ...        5000.0           5.0   
284028     Alteneiji80          1.0  ...        3200.0           4.0   
277641     Alteneiji80          1.0  ...        1505.0           3.0   

        TotalCashouts  EOM_Equity_Copy  EOM_Equity_Real_Crypto  \
217755            0.0             0.00                     0.0   
217753            0.0             0.00                     0.0   
217754            0.0             0.00                     0.0   
217752            0.0             0.00                     0.0   
219842            0.0             0.00                     0.0   
...               ...              ...                     ...   
277643            0.0             0.00                     0.0   
277642            0.0             0.00                     0.0   
277640         3910.0             0.00                     0.0   
284028            0.0             0.00                     0.0   
277641            0.0          1222.06                     0.0   

        EOM_Equity_Real_Stocks  EOM_Equity_CFD_Crypto  EOM_Equity_CFD_Stocks  \
217755                     0.0                   0.00                    0.0   
217753                     0.0                   0.00                    0.0   
217754                     0.0                   0.00                    0.0   
217752                     0.0                   0.00                    0.0   
219842                     0.0                   0.00                    0.0   
...                        ...                    ...                    ...  

In [97]:
len(exclude)

481

In [45]:
cols = ['ID', 'ActiveDate', 'EOM_Equity', TARGET_COL] 
rawdata[cols][rawdata.ID=='0000FD14-7FA7-EA11-A2D7-0017A4770404']



ID ActiveDate  EOM_Equity    target
277647  0000FD14-7FA7-EA11-A2D7-0017A4770404 2021-02-28    57805.04       NaN
277646  0000FD14-7FA7-EA11-A2D7-0017A4770404 2021-01-31    53948.83  57805.04
277645  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-12-31    18453.84  53948.83
277644  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-11-30    51364.17  18453.84
277643  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-10-31    15415.98  51364.17
277642  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-09-30    14553.37  15415.98
277640  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-08-31     2824.25  14553.37
284028  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-07-31     5248.30   2824.25
277641  0000FD14-7FA7-EA11-A2D7-0017A4770404 2020-06-30     1312.92   5248.30